In [ ]:
import latticeproteins as lp # envt is latticeproteins
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import binom
from helpers import DotAccessibleDict
import multiprocessing as mp

!pwd

In [ ]:
# plot distribution of xshift and yshift
seq_length = 12
xshifts = []
yshifts = []
num=10000
for i in range(num):
    lattice = lp.thermodynamics.LatticeThermodynamics.from_length(seq_length, 1.0)
    conf = None
    while conf is None:
        seq = lp.random_sequence(seq_length)
        conf = lattice.native_conf(seq)
    #print(seq, conf)

    lig_length = 6
    lig_lattice = lp.thermodynamics.LatticeThermodynamics.from_length(lig_length, 1.0)
    lig_conf = None
    while lig_conf is None:
        lig_seq = lp.random_sequence(lig_length)
        lig_conf = lig_lattice.native_conf(lig_seq)
    #print(lig_seq, lig_conf)

    (be, xshift, yshift, lig_conf) = lp.conformations.BindLigand(seq, conf, lig_seq, lig_conf)
    ligand_tup = (lig_seq, None, lig_conf, xshift, yshift) # None is for ligand_color_seq
    #print(f"be: {be:0.3f}")
    #print(f"xshift: {xshift}, yshift: {yshift}")
    #lp.draw.in_notebook(seq, conf, ligand_tup=ligand_tup)
    xshifts.append(xshift)
    yshifts.append(yshift)

plt.figure(figsize=(6,6))
# integer-centered bin edges
xmin, xmax = min(xshifts), max(xshifts)
ymin, ymax = min(yshifts), max(yshifts)
x_edges = np.arange(xmin - 0.5, xmax + 1.5, 1.0)
y_edges = np.arange(ymin - 0.5, ymax + 1.5, 1.0)

h = plt.hist2d(xshifts, yshifts, bins=[x_edges, y_edges], cmap='Blues')
plt.colorbar(h[3], label='counts')
plt.xlabel("xshift")
plt.ylabel("yshift")
plt.title(f"2D histogram of ligand binding position shifts\nover {num} random proteins and ligands")
plt.axhline(0, color='gray', linestyle='--')
plt.axvline(0, color='gray', linestyle='--')
plt.gca().set_aspect('equal')#, adjustable='box')
plt.show()

In [ ]:
## Fold lattice protein to 5 nonnative states

from IPython.display import HTML, SVG, display

# Find the 5 lowest energy conformations.
alt_conf = lattice.conformations.k_lowest_confs(seq, 1.0, 5)

for i, conf in enumerate(alt_conf): # target conformation

    (be, xshift, yshift, lig_conf) = lp.conformations.BindLigand(seq, conf, lig_seq, lig_conf)
    ligand_tup = (lig_seq, None, lig_conf, xshift, yshift)
    print(f"be: {be:0.3f}")
    
    print("seq:", seq, "conf:", conf)
    print("Energy of native conformation: %f" % lattice.nativeE(seq, target=conf))
    print("stability of native conformation: %f" % lattice.stability(seq, target=conf))
    print("fraction folded: %f" % lattice.fracfolded(seq, target=conf))
    svg = lp.draw.Configuration(seq, conf, ligand_tup=ligand_tup)
    display(SVG(data=svg.data))
    if i < len(alt_conf) - 1:
        print("\n----------------------\n")


lp.sequences.n_mutants(seq, 1, 1)
 